# WorkGraph example to run Molecular Dynamics

## Aim

As an example, we start from a structure, run an md simulation, compute descriptors, and then use a filtering function to split the resulting structures into `train.xyz`, `test.xyz`, and `valid.xyz`.

Load the aiida profile, structure, model and code:

In [ ]:
from aiida import load_profile
load_profile()

In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [ ]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read, iread

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
init_structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [ ]:
from aiida.orm import load_code
janus_code = load_code("janus@localhost")

In [ ]:
from aiida.orm import Str, Float, Bool, Int, Dict
inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "ensemble": Str("NVT"),
    "struct": init_structure,
    "md_kwargs": Dict(
        {
            "steps": 10,
            "traj-every": 2
        }
    )
}

In [ ]:
from aiida.plugins import CalculationFactory

mdCalc = CalculationFactory("mlip.md")
descriptorsCalc = CalculationFactory("mlip.descriptors")

# Single Descriptor

In [ ]:
from aiida_workgraph import WorkGraph
wg = WorkGraph("MD_workgraph")
md_calc = wg.add_task(
    mdCalc,
    name="md_calc",
    **inputs
)


In [ ]:
# To find inputs/outputs of mdcalc uncomment following: 
# mdCalc.get_description()["spec"]["outputs"].keys()

In [ ]:
descriptors_calc = wg.add_task(
    descriptorsCalc,
    name="descriptors_calc",
    struct=md_calc.outputs.final_structure,
)

In [ ]:
from sample_split import process_and_split_data

split_data = wg.add_task(
    process_and_split_data,
    name="split_data",
    struct=descriptors_calc.outputs.xyz_output
)


In [ ]:
wg

In [ ]:
wg.tasks

In [ ]:
# wg.run()

# Loop Descriptor

In [ ]:
from aiida_workgraph import WorkGraph

with WorkGraph("MD_Simple") as wg:
    
    # MD simulation
    md_task = wg.add_task(
        mdCalc,
        name="md",
        **inputs
    )
 
    
    # Descriptors on final structure
    desc_task = wg.add_task(
        descriptorsCalc,
        name="descriptors",
        code=inputs['code'],
        model=inputs['model'],
        arch=inputs['arch'],
        device=inputs['device'],
        metadata=inputs['metadata'],
        calc_per_element=Bool(True),
        struct=md_task.outputs.final_structure
    )

wg.run()

In [ ]:
wg